# add c_EC as one of the inputs for diffusivity and conductivity; use exponeitial extrapolation for Nyman2008 and make the added function a tanh, use andrew properties

In [15]:
import pybamm
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd   ;import numpy as np;import os;import matplotlib.pyplot as plt;import os;
from scipy.io import savemat,loadmat;from pybamm import constants,exp,tanh,sqrt;import matplotlib as mpl; fs=17; # or we can set import matplotlib.pyplot as plt then say 'mpl.rc...'
for k in range(0,1):
    mpl.rcParams["axes.labelsize"] = fs
    mpl.rcParams["axes.titlesize"] = fs
    mpl.rcParams["xtick.labelsize"] =  fs
    mpl.rcParams["ytick.labelsize"] =  fs
    mpl.rcParams["legend.fontsize"] =  fs
    mpl.rcParams['font.sans-serif'] = ['Times New Roman']
    mpl.rcParams['font.serif'] = ['Times New Roman']
    mpl.rcParams['axes.unicode_minus'] = False
import traceback

In [16]:
# define function:
def electrolyte_conductivity_Nyman2008_wEC(c_e,c_EC, T):
    x = c_EC / c_e
    coff = 1
    sigma_e = (
        (0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2)) * 
        (
        0.1 * 0.06248 * (1+298.15-0.05559) * 
        (c_e/1e3) * (1 - 3.084 *sqrt(c_e/1e3) 
        + 1.33 *(1+ 0.03633 *(exp(1000/298.15))*c_e/1e3)   ) 
        / (1+(c_e/1e3)**4*( 0.00795 *exp(1000/298.15))) )  )
    return sigma_e
def electrolyte_diffusivity_Nyman2008_wEC(c_e, c_EC,T):
    x = c_EC / c_e
    coff = 1
    D_c_e = (
        ( 0.00+ coff/2 + coff/2 *  tanh((x-2.7705)*2) )  * 
        (6 * exp( -1 *(c_e/1000)) 
        * exp(-5/298.15) 
        * exp(-95/298.15*(c_e/1000)) * 1e-10 ) )
    return D_c_e


In [17]:
V_max = 4.2;        V_min = 2.5;
Diff_SEI = 1.7e-20;    R_SEI = 2E5;   Bulk_Sol_Con = 4541.0;
Experiment_Long   = pybamm.Experiment( [ (
    f"Discharge at 1 C until {V_min} V", 
    f"Charge at 0.3 C until {V_max} V", 
    f"Hold at {V_max} V until C/5")    ] * 1 ) 
Experiment_s   = pybamm.Experiment( [ (
    f"Discharge at 1 C for 240 s", )    ] * 1 )   

In [18]:
BasicPath = 'D:/OneDrive - Imperial College London/SimDataSave/P3R4/'; 
Target  = 'a2_Diff_Cond_c_EC/'
if not os.path.exists(BasicPath + Target):
    os.mkdir(BasicPath + Target);

In [19]:
Model_ddiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "EC wo refill"     } )
Model_sdiff = pybamm.lithium_ion.DFN(     
    options={"solvent diffusion": "none"     } )
ChemistryChen=pybamm.parameter_sets.Chen2020 
ChemistryChen["electrolyte"] = "lipf6_Nyman2008_rio";
Para_0=pybamm.ParameterValues(chemistry=ChemistryChen);


Para_0['EC Lithium ion cross diffusivity [m2.s-1]'] = "[input]" # 1.5e-9/1e2;
Para_0['Typical EC Lithium ion cross diffusivity [m2.s-1]'] =   1.5e-9/1e2
Para_0['EC transference number'] =     "[input]" # -1.4
Para_0['EC diffusivity in electrolyte [m2.s-1]'] =   "[input]" # 5E-12;
Para_0.update(
    {'Electrolyte diffusivity [m2.s-1]':
    3e-10})
Para_0.update(
    {'Electrolyte conductivity [S.m-1]':
    electrolyte_conductivity_Nyman2008_wEC})
c_e_ddiff = Model_ddiff.variables["Electrolyte concentration [mol.m-3]"]
T_ddiff = Model_ddiff.variables["Cell temperature [K]"]
c_EC_ddiff = Model_ddiff.variables["EC concentration [mol.m-3]"]
Model_ddiff.variables["c(EC) over c(Li+)"] = c_EC_ddiff / c_e_ddiff
Model_ddiff.variables["Electrolyte conductivity [S.m-1]"] =electrolyte_conductivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)
Model_ddiff.variables["Electrolyte diffusivity [m2.s-1]"] =electrolyte_diffusivity_Nyman2008_wEC(c_e_ddiff,c_EC_ddiff, T_ddiff)

using EC wo refill for Li+


Succeed for:  Xi= -1.8 t_0plus= 0.01 D_cross= 1.5e-18 D_ec= 3.750125e-10

In [26]:
Sol_ddiff    = Sim_ddiff.solve(
    inputs={
        "EC transference number":-1.35,
        "Cation transference number":0.28,
        "EC Lithium ion cross diffusivity [m2.s-1]":0,
        "EC diffusivity in electrolyte [m2.s-1]":5e-10,});

The linesearch algorithm failed with too small a step.


In [ ]:
print(Set_succ)

In [ ]:
Sim_ddiff    = pybamm.Simulation(
    Model_ddiff, experiment = Experiment_Long,
    parameter_values = Para_0,
    solver = pybamm.CasadiSolver(),)   
index_c = 5
Xi =  list(np.linspace(-1.5, -1.3,index_c));
t_0plus = list(np.linspace(0.01, 0.3, index_c));
D_cross = list(np.linspace(0, 1.5e-9,9));
D_ec = list(np.linspace(5e-14, 5e-10, index_c));
Set_succ = []; Set_fail = []; ErrorMess = [];
t_0plus_i = 0.28;  D_ec_i = 5e-10; 
for Xi_i in Xi:
    for D_cross_i in D_cross:
            try:
                Sol_ddiff    = Sim_ddiff.solve(
                    inputs={
                        "EC transference number":Xi_i,
                        "Cation transference number":t_0plus_i,
                        "EC Lithium ion cross diffusivity [m2.s-1]":D_cross_i,
                        "EC diffusivity in electrolyte [m2.s-1]":D_ec_i,});
            except:
                str_error = traceback.format_exc()  
                print(str_error)
                ErrorMess.append(str_error)
                print("Fail for: ", 
                'Xi=',Xi_i, "t_0plus=",t_0plus_i, 
                "D_cross=",D_cross_i, "D_ec=", D_ec_i );
                Set_fail.append([Xi_i,t_0plus_i,D_cross_i,D_ec_i])

            else:
                print("Succeed for: ", 
                'Xi=',Xi_i, "t_0plus=",t_0plus_i, 
                "D_cross=",D_cross_i, "D_ec=", D_ec_i );
                Set_succ.append([Xi_i,t_0plus_i,D_cross_i,D_ec_i])
            

In [ ]:
Model_ddiff.variables.search("potential")

In [ ]:
print(Sol_ddiff["Electrolyte conductivity [S.m-1]"].entries[:,51] )
print(Sol_ddiff["Electrolyte diffusivity [m2.s-1]"].entries[:,-1])

In [27]:
font = {'family' : 'Times New Roman',
        #'weight' : 'bold',
        'size'   : 18}
mpl.rc('font', **font)

In [28]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"fixed_add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=3.25169569451806, step=0.0325169569451806), …

In [29]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    ["EC flux [mol.m-2.s-1]","EC flux by diffusion [mol.m-2.s-1]",
        "EC flux by migration [mol.m-2.s-1]","EC flux by Li+ [mol.m-2.s-1]"],
    ["Li+ flux [mol.m-2.s-1]","Li+ flux by diffusion [mol.m-2.s-1]",
        "Li+ flux by migration [mol.m-2.s-1]","Li+ flux by solvent [mol.m-2.s-1]"],    
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=1,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();

interactive(children=(FloatSlider(value=0.0, description='t', max=3.25169569451806, step=0.0325169569451806), …

In [30]:
label = ["Sol_ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    "Electrolyte potential [V]",
    "Positive electrode potential [V]",
    "Negative electrode SOC",
    "Positive electrode SOC",
    "Porosity times concentration",
    "eps_c_e rhs",
    "Minus div Li+ flux",
    "Li+ source term",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff,
    ], output_variables3,label,
    variable_limits='fixed',time_unit='hours',n_rows=3,
    figsize = (16,12)) #     spatial_unit='mm',
quick_plot.dynamic_plot();


interactive(children=(FloatSlider(value=0.0, description='t', max=3.25169569451806, step=0.0325169569451806), …

In [31]:
label = ["ddiff"] 
output_variables3 = [
    "Terminal voltage [V]",   
    #"Current [A]",
    "EC concentration [mol.m-3]",
    "Electrolyte concentration [mol.m-3]",
    "c(EC) over c(Li+)",
    #"Loss of capacity to SEI [A.h]",
    "Li+ flux [mol.m-2.s-1]",
    "EC flux [mol.m-2.s-1]",
    "Electrolyte conductivity [S.m-1]",
    "Electrolyte diffusivity [m2.s-1]",
]
quick_plot = pybamm.QuickPlot([
    Sol_ddiff
    ], output_variables3,label,
    variable_limits='tight',time_unit='hours',n_rows=2,
    figsize = (16,6)) #     spatial_unit='mm',
quick_plot.dynamic_plot();
#quick_plot.create_gif(
#    number_of_images=10, duration=2,output_filename=BasicPath + Target+"add c_EC as tanh (0-1).gif")

interactive(children=(FloatSlider(value=0.0, description='t', max=3.25169569451806, step=0.0325169569451806), …